In [1]:
import numpy as np
import pandas as pd
import sys

from numpy.random import normal

import cobra

sys.path.append('../')
from uncBiomFuncs import file_name, make_normal_distributed_coefficients, make_pfba_csv, make_fluxes_dataframe 

In [17]:
coeff_std = 0.1 # Set this to 0.05, 0.1, 0.2, and 0.3 and rerun
size = 50

In [18]:
dir_model = '../input/iML1515_glcreduced.json'
model = cobra.io.load_json_model(dir_model)
model.solver = 'cplex'

revBetaOx = ['ACACT1r', 'ACACT2r', 'ACACT3r', 'ACACT4r', 'ACACT5r',
             'ACACT6r', 'ACACT7r', 'ACACT8r']
for rxnid in revBetaOx:
    rxn = model.reactions.get_by_id(rxnid)
    rxn.upper_bound = 0
model.reactions.POR5.lower_bound = 0

config = cobra.Configuration()
config.tolerance = 1e-9
zero_tol = 1e-9

biomId = 'BIOMASS_Ec_iML1515_core_75p37M'
gam_val = 75.37723
dir_case = './output/'
import os
if not os.path.exists(dir_case):
    os.makedirs(dir_case)

In [19]:
# Step 1 - vary coefficient
fname = file_name(coeff_std=coeff_std, gam_std=None, ftype='dfCoeff')
dfCoeff = make_normal_distributed_coefficients(model, coeff_std=coeff_std,
                gam_val=gam_val, biomId=biomId, size=size)
dfCoeff.to_csv(dir_case + fname + '.csv', sep='\t')

In [20]:
# Step 2 - run pFBA
fname = file_name(coeff_std=coeff_std, gam_std=None, ftype='dfCoeff')
dfCoeff = pd.read_csv(dir_case + fname + '.csv', sep='\t', index_col=0, header=0)

fname = file_name(coeff_std=coeff_std, gam_std=None)
dir_pfba = dir_case + 'pFBA/' + fname + '/'

dir_model = '../input/iML1515_glcreduced.json'
model = cobra.io.load_json_model(dir_model)
model.solver = 'cplex'

revBetaOx = ['ACACT1r', 'ACACT2r', 'ACACT3r', 'ACACT4r', 'ACACT5r',
             'ACACT6r', 'ACACT7r', 'ACACT8r']
for rxnid in revBetaOx:
    rxn = model.reactions.get_by_id(rxnid)
    rxn.upper_bound = 0
model.reactions.POR5.lower_bound = 0

config = cobra.Configuration()
config.tolerance = 1e-9
zero_tol = 1e-9

make_pfba_csv(model, dfCoeff, dir_pfba, biomId)

No infeasible cases


In [21]:
# Step 3 - compile flux dataframe from pFBA csv
fname = file_name(coeff_std=coeff_std, gam_std=None, ftype='dfCoeff')
dfCoeff = pd.read_csv(dir_case + fname + '.csv', sep='\t', index_col=0, header=0)

fname = file_name(coeff_std=coeff_std, gam_std=None)
dir_pfba = dir_case + 'pFBA/' + fname + '/'

fname = file_name(coeff_std=coeff_std, gam_std=None, ftype='dfFlux')
index = dfCoeff.index.to_list()
dfFlux = make_fluxes_dataframe(index, dir_pfba)
dfFlux = dfFlux.loc[:, (dfFlux.abs() > zero_tol).any(axis=0)]
dfFlux.to_csv(dir_case + fname + '.csv', sep='\t')

# Remove pFBA files
import shutil
fname = file_name(coeff_std=coeff_std, gam_std=None)
shutil.rmtree(dir_pfba)